In [47]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [161]:
# https://gymnasium.farama.org/introduction/basic_usage/
# https://gymnasium.farama.org/environments/box2d/

import gymnasium as gym
import gymnasium_env
from gymnasium.wrappers import FlattenObservation
import keyboard
import numpy as np
from utils import *

#### Generate data for training BC imitation learning model

 - Data generaiton functions in `utils.py`.

In [131]:
env = gym.make('gymnasium_env/GridWorld-v0', render_mode="human")
observation, info = env.reset()
observation

{'agent': array([2, 2]), 'target': array([1, 4])}

In [146]:
observation={'agent': np.array([2, 1]), 'target': np.array([1, 0])}


In [147]:
%%time
# generate actions
actions=compute_min_steps_to_win(observation, shuffle=False, print_values=True)
actions

Agent: [2 1] Target: [1 0]
Difference: [-1 -1]
XY Actions: [2, 1]
CPU times: total: 0 ns
Wall time: 0 ns


[2, 1]

In [148]:
%%time
# generate positions
positions=generate_agent_target_positions(observation, actions)
positions

CPU times: total: 0 ns
Wall time: 0 ns


{0: {'agent': array([2, 1]), 'target': array([1, 0]), 'action': -1},
 1: {'agent': array([1, 1]), 'target': array([1, 0]), 'action': 2},
 2: {'agent': array([1, 0]), 'target': array([1, 0]), 'action': 1}}

In [159]:
%%time
# reconstruct grid per position
# ts=2
# agent=positions[ts]["agent"]
# target=positions[ts]["target"]
# reconstruct_grid(agent, target, observation=None, size=5)
grid_data=generate_grid_action_dict(positions, size=5)
print(grid_data)

[{'grid': [0, 1, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'action': -1}, {'grid': [0, 1, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'action': 2}, {'grid': [0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'action': 1}]
CPU times: total: 0 ns
Wall time: 0 ns


In [163]:
# iteratively generate training data
training_data=generate_training_data(env, N=1, shuffle=False, print_values=False, size=5)

100%|██████████| 1/1 [00:00<00:00, 285.35it/s]


{0: [{'grid': [0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    1,
    0,
    0,
    0,
    2,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0],
   'action': -1},
  {'grid': [0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    1,
    0,
    0,
    2,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0],
   'action': 2},
  {'grid': [0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    1,
    0,
    2,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0],
   'action': 2},
  {'grid': [0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    1,
    2,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0],
   'action': 2},
  {'grid': [0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    3,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0],
   'acti

#### Testing env

In [6]:
env = gym.make('gymnasium_env/GridWorld-v0', render_mode="human")
observation, info = env.reset()

# done = False
# print("Controls: W (up), S (down), A (left), D (right), Q (quit)")
# while not done:
#     env.render() 
    
#     action = None
#     while action is None:
#         if keyboard.is_pressed("w"):
#             action = 0  # Up
#         elif keyboard.is_pressed("s"):
#             action = 1  # Down
#         elif keyboard.is_pressed("a"):
#             action = 2  # Left
#         elif keyboard.is_pressed("d"):
#             action = 3  # Right
#         elif keyboard.is_pressed("q"):
#             print("Exiting...")
#             env.close()
#             exit(0)

#     observation, reward, done, truncated, info = env.step(action)
#     print(f"Reward: {reward}")

# env.close()

#### Scratch code

In [15]:
observation, reward, terminated, truncated, info=env.step(env.action_space.sample())

In [21]:
env.step(env.action_space.sample())

({'agent': array([0, 1]), 'target': array([2, 4])},
 0,
 False,
 False,
 {'distance': 5.0})

In [19]:
# example env wrapper
wrapped_env = FlattenObservation(env)
print(wrapped_env.reset())     # E.g.  [3 0 3 3], {}

(array([3, 1, 4, 4]), {'distance': 4.0})


In [25]:
observation, info = env.reset()

In [26]:
observation

{'agent': array([4, 0]), 'target': array([0, 2])}